In [1]:
import tensorflow as tf
from tensorflow.keras.models import model_from_json, Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import os

2025-02-14 20:12:57.479121: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-14 20:12:58.430052: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/cuda-11.8/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda-12.6/lib64
2025-02-14 20:12:58.430147: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7

In [2]:
# -----------------------------
# Step 1: Load the Pre-trained Model (Keras Format)
# -----------------------------

# Define paths for the architecture and weights files
arch_path = "/home/natalyagrokh/img_expressions/final_efficientnet_trained_model_2_architecture.json"
weights_path = "/home/natalyagrokh/img_expressions/final_efficientnet_trained_model_2_weights.h5"

# Load the model architecture from the JSON file.
with open(arch_path, "r") as json_file:
    loaded_model_json = json_file.read()
loaded_model = model_from_json(loaded_model_json)
print("Model architecture loaded from JSON.")

# Load the model weights from the H5 file.
loaded_model.load_weights(weights_path)
print("Model weights loaded from H5.")
print("Model re-loaded successfully from JSON and H5 files.")

2025-02-14 20:13:00.555598: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-14 20:13:00.596169: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-14 20:13:00.598368: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-14 20:13:00.601349: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Model architecture loaded from JSON.
Model weights loaded from H5.
Model re-loaded successfully from JSON and H5 files.


In [3]:
# -----------------------------
# Step 2: Use the Pre-trained Model as a Base
# -----------------------------
# Use the loaded model (from JSON/H5) as the pre-trained model.
base_output = loaded_model.layers[-2].output  # Here, '-2' refers to the output before the final classification layer

# Create a new base model using the original input and the selected output.
base_model = Model(inputs=loaded_model.input, outputs=base_output)
print("Base model constructed from pre-trained model (without final classification layer).")

# Freeze all layers in the base model so that training on fer2013 initially doesn't modify these weights.
for layer in base_model.layers:
    layer.trainable = False

Base model constructed from pre-trained model (without final classification layer).


In [4]:
# -----------------------------
# Step 3: Build a New Classification Head for fer2013
# -----------------------------
# fer2013 has 7 emotion classes (the same number as AffectNet in your case),
# but you might still benefit from reinitializing the head for better adaptation.
new_head = Sequential([
    # (Optional) Add a Dense layer for further processing.
    Dense(512, activation="relu"),
    Dropout(0.3),
    # Final classification layer for 7 classes.
    Dense(7, activation="softmax")
], name="new_head")

In [5]:
# -----------------------------
# Step 4: Combine the Base and New Head into a New Model
# -----------------------------
# Create an input tensor matching your fer2013 data shape.
input_tensor = Input(shape=(224, 224, 3))
# Add a resizing layer to convert the input to the base model’s expected shape.
resized_tensor = tf.keras.layers.Resizing(48, 48)(input_tensor)
# Pass the resized input through the base model.
x = base_model(resized_tensor)
# Then pass through the new head.
output_tensor = new_head(x)
# Build the final model.
model_fer = Model(inputs=input_tensor, outputs=output_tensor)
print("New model for fer2013 created.")

New model for fer2013 created.


In [6]:
# -----------------------------
# Step 5: Compile the Model for Fine-Tuning on fer2013
# -----------------------------
# Use a relatively low learning rate since we are fine-tuning.
optimizer = Adam(learning_rate=1e-4)
model_fer.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"]
)
model_fer.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resizing (Resizing)         (None, 48, 48, 3)         0         
                                                                 
 model (Functional)          (None, 128)               4213539   
                                                                 
 new_head (Sequential)       (None, 7)                 69639     
                                                                 
Total params: 4,283,178
Trainable params: 69,639
Non-trainable params: 4,213,539
_________________________________________________________________


In [7]:
# -----------------------------
# Step 6: Set Up Callbacks and Data Generators (as in your base code)
# -----------------------------
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# (Reuse your existing data augmentation code for fer2013)
train_aug_gen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rescale=1/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_gen = ImageDataGenerator(rescale=1/255)

batch_size = 2
image_shape = (224, 224, 3)
train_dir = '/home/natalyagrokh/img_datasets/fer_2013_dataset/train'
test_dir = '/home/natalyagrokh/img_datasets/fer_2013_dataset/test'

train_image_gen = train_aug_gen.flow_from_directory(
    train_dir,
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb'
)
test_image_gen = val_gen.flow_from_directory(
    test_dir,
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb'
)
# Define callbacks (adjust patience and learning rate factors as needed)
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=0.01,
    restore_best_weights=True,
    verbose=1
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.7,
    min_delta=0.005,
    patience=5,
    verbose=1
)

steps_per_epoch = max(1, train_image_gen.samples // batch_size)
validation_steps = max(1, test_image_gen.samples // batch_size)
print("Batch size set to:", batch_size)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Batch size set to: 2


In [8]:
# Disable fused batch normalization to force the use of non-fused (slower, but more robust) BN kernels.
os.environ["TF_ENABLE_FUSED_BN"] = "0"
print("Set TF_ENABLE_FUSED_BN to 0 (disabled).")

Set TF_ENABLE_FUSED_BN to 0 (disabled).


In [9]:
# -----------------------------
# Step 8: Train the Model on fer2013
# -----------------------------
# (Optional) Ensure soft device placement is enabled.
tf.config.set_soft_device_placement(True)

# -----------------------------
# Step 7: Train the Model on fer2013
# -----------------------------
history = model_fer.fit(
    train_image_gen,
    epochs=100,  # Adjust epochs as needed
    steps_per_epoch=steps_per_epoch,
    validation_data=test_image_gen,
    validation_steps=validation_steps,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/100


2025-02-14 20:13:15.911699: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/model/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2025-02-14 20:13:16.425035: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8907
2025-02-14 20:13:16.665017: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x56459e64ef00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-02-14 20:13:16.665053: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-02-14 20:13:16.671158: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-02-14 20:13:16.809075: I tensorflow/compiler/jit/xla_compilation_cache

14354/14354 [==============================] - 390s 27ms/step - loss: 1.8366 - accuracy: 0.2656 - val_loss: 1.6597 - val_accuracy: 0.3862 - lr: 1.0000e-04
Epoch 2/100
14354/14354 [==============================] - 381s 27ms/step - loss: 1.7798 - accuracy: 0.2888 - val_loss: 1.6855 - val_accuracy: 0.3816 - lr: 1.0000e-04
Epoch 3/100
14354/14354 [==============================] - 381s 27ms/step - loss: 1.7635 - accuracy: 0.3024 - val_loss: 1.6883 - val_accuracy: 0.3858 - lr: 1.0000e-04
Epoch 4/100
14354/14354 [==============================] - 376s 26ms/step - loss: 1.7599 - accuracy: 0.2952 - val_loss: 1.6853 - val_accuracy: 0.3906 - lr: 1.0000e-04
Epoch 5/100
14354/14354 [==============================] - 376s 26ms/step - loss: 1.7537 - accuracy: 0.3032 - val_loss: 1.6681 - val_accuracy: 0.3859 - lr: 1.0000e-04
Epoch 6/100
14352/14354 [============================>.] - ETA: 0s - loss: 1.7603 - accuracy: 0.3021       
Epoch 6: ReduceLROnPlateau reducing learning rate to 6.99999982316512

In [ ]:
# (Optional) Print GPU information to verify detection and memory
gpus = tf.config.list_physical_devices('GPU')
print("GPUs available:", gpus)
for gpu in gpus:
    try:
        details = tf.config.experimental.get_device_details(gpu)
        print(f"Details for {gpu.name}:", details)
    except Exception as e:
        print(f"Error retrieving details for {gpu}: {e}")

In [9]:
print(tf.sysconfig.get_build_info())

OrderedDict([('cpu_compiler', '/dt9/usr/bin/gcc'), ('cuda_compute_capabilities', ['sm_35', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'compute_80']), ('cuda_version', '11.2'), ('cudnn_version', '8'), ('is_cuda_build', True), ('is_rocm_build', False), ('is_tensorrt_build', True)])
